In [49]:
import xmltodict
import requests
import json

In [50]:
url_road = \
    'http://apis.data.go.kr/1611000/AptListService/getRoadnameAptList?' \
    'roadCode=263802006002' \
    '&ServiceKey=aT8%2FTBG8hGkv%2BYtyPqc%2FPhTFkLeNJqNqITQu120BvseWmbAqVhHLN7XB3tijd471W9%2Fy8Zbx6sVNgvoPXjtUSw%3D%3D'

url_bjd = \
    'http://apis.data.go.kr/1611000/AptListService/getLegaldongAptList?' \
    'bjdCode=1120011400' \
    '&ServiceKey=V8giduxGZ%2BU463maB552xw3jULhTVPrv%2B7m2qSqu4w8el9fk8bnMD9i6rjUQz7gcUcFnDKyOmcCBztcbVx3Ljg%3D%3D'

# response = requests.get(url_bjd)


In [51]:
response = requests.get(url_bjd).content
xmlObj = xmltodict.parse(response)
json_data = json.dumps(xmlObj, ensure_ascii=False)
dict_data = json.loads((json_data))


In [52]:
dict_data

{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'},
  'body': {'items': {'item': [{'kaptCode': 'A13382502', 'kaptName': '성수현대'},
     {'kaptCode': 'A13311101', 'kaptName': '성수금호3차'},
     {'kaptCode': 'A13384301', 'kaptName': '한양현대아파트'},
     {'kaptCode': 'A13372102', 'kaptName': '성수쌍용'},
     {'kaptCode': 'A13379401', 'kaptName': '성수갤러리아포레'},
     {'kaptCode': 'A13372101', 'kaptName': '성수2차대우'},
     {'kaptCode': 'A13376405', 'kaptName': '성수한진타운'},
     {'kaptCode': 'A13384304', 'kaptName': '성수동아그린'},
     {'kaptCode': 'A13392307', 'kaptName': '강변건영'},
     {'kaptCode': 'A13384303', 'kaptName': '뚝섬중앙하이츠빌'}]},
   'numOfRows': '10',
   'pageNo': '1',
   'totalCount': '14'}}}

In [53]:
data = dict_data['response']['body']['items']['item']

In [54]:
data

[{'kaptCode': 'A13382502', 'kaptName': '성수현대'},
 {'kaptCode': 'A13311101', 'kaptName': '성수금호3차'},
 {'kaptCode': 'A13384301', 'kaptName': '한양현대아파트'},
 {'kaptCode': 'A13372102', 'kaptName': '성수쌍용'},
 {'kaptCode': 'A13379401', 'kaptName': '성수갤러리아포레'},
 {'kaptCode': 'A13372101', 'kaptName': '성수2차대우'},
 {'kaptCode': 'A13376405', 'kaptName': '성수한진타운'},
 {'kaptCode': 'A13384304', 'kaptName': '성수동아그린'},
 {'kaptCode': 'A13392307', 'kaptName': '강변건영'},
 {'kaptCode': 'A13384303', 'kaptName': '뚝섬중앙하이츠빌'}]

In [57]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, UnexpectedAlertPresentException
import os
import re
import time
import urllib


In [58]:
driver = webdriver.Chrome('/Users/mac/projects/ChromeWebDriver/chromedriver')


In [59]:
url = 'https://www.dabangapp.com/room/5e4b99581a00fb457ff0ef63'

In [60]:
driver.get(url)

In [61]:
moveIn = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div/ul/li[9]/div')

In [62]:
moveIn = moveIn.get_attribute('innerText')

In [63]:
moveIn

'2020.03.23'

In [64]:
test_url = 'https://www.dabangapp.com/room/5e9e3d85da8873165c0640f4'
driver.get(test_url)

In [65]:
test = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div/ul/li[9]/div')
test = test.get_attribute('innerText')

In [67]:
print(type(test))

<class 'str'>


In [68]:
date = '날짜 협의'
print(type(date))

<class 'str'>


In [71]:
if '날짜' in moveIn:
    print('날짜 있음')
elif '즉시' in moveIn:
    prnt("즉시 입주")
else: 
    print(moveIn)

2020.03.23


In [77]:
moveIn = '2020.03.23'

In [79]:
if '날짜' in moveIn:
    print('날짜 있음')
elif '즉시' in moveIn:
    print("즉시 입주")
else: 
    moveIn= moveIn.replace('.', '-')

In [80]:
moveIn

'2020-03-23'

In [81]:
url = 'https://www.dabangapp.com/room/5e94139c0eb4bb3b5f8d29c4'
driver.get(url)

parking = driver.find_element_by_xpath('/html/body/div[1]/div/div[5]/div[2]/div/table/tbody/tr/td[4]/p')

In [83]:
parking = parking.get_attribute('innerText')

In [85]:
parking

'3만 원'